In [5]:
import torch
from transformers import pipeline

In [11]:

classifier = pipeline(
    model = "distilbert-base-uncased-finetuned-sst-2-english"
)

result = classifier("It's definitely a movie")
print(result)

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9998583793640137}]
